In [1]:
from PyQt5 import QtWidgets
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import sys
import cv2 # OpenCV
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
import glob
import re
import os
from functools import partial
import matplotlib.image as mpimg

In [2]:
#code to make files run in numerical order
numbers = re.compile(r'(\d+)')
def numericalSort(val):
    parts = numbers.split(val)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [3]:
#setting up the next window with buttons and cropped pictures
class AnotherWindow(QWidget):
    def __init__(self):
        super().__init__()
#         lay = QVBoxLayout()
        #setting up the size of window
        self.setGeometry(400,300,800,450)
        self.label = QLabel(self)
        
        #setting up six buttons with text, location, and what to do when clicked
        #get text of clicked button
        self.b1 = QtWidgets.QPushButton(self)
        self.b1.setText("1) MTCNN \n2) HOG\n3) Haar")
        self.b1.move(80,340)
        self.b1.clicked.connect(partial(self.save, self.b1))
        self.b1.clicked.connect(lambda:self.getText(self.b1))
                
        self.b2 = QtWidgets.QPushButton(self)
        self.b2.setText("1) MTCNN \n2) Haar\n3) HOG")
        self.b2.move(190,340)
        self.b2.clicked.connect(partial(self.save,self.b2))
        self.b2.clicked.connect(lambda:self.getText(self.b2))

        self.b3 = QtWidgets.QPushButton(self)
        self.b3.setText("1) Haar \n2) MTCNN\n3) HOG")
        self.b3.move(300,340)
        self.b3.clicked.connect(partial(self.save,self.b3))
        self.b3.clicked.connect(lambda:self.getText(self.b3))

        self.b4 = QtWidgets.QPushButton(self)
        self.b4.setText("1) Haar \n2) HOG\n3) MTCNN")
        self.b4.move(410,340)
        self.b4.clicked.connect(partial(self.save,self.b4))
        self.b4.clicked.connect(lambda:self.getText(self.b4))

        self.b5 = QtWidgets.QPushButton(self)
        self.b5.setText("1) HOG \n2) MTCNN\n3) Haar")
        self.b5.move(520,340)
        self.b5.clicked.connect(partial(self.save,self.b5))
        self.b5.clicked.connect(lambda:self.getText(self.b5))

        self.b6 = QtWidgets.QPushButton(self)
        self.b6.setText("1) HOG \n2) Haar\n3) MTCNN")
        self.b6.move(630,340)
        self.b6.clicked.connect(partial(self.save,self.b6))
        self.b6.clicked.connect(lambda:self.getText(self.b6))

        
        key = 'mapped'
        m = 'MTCNN'
        m1 = 'MTCNN 1'
        ho = 'HOG'
        ho1 = 'HOG 1'
        ha = 'Haar'
        ha1 = 'Haar 1'
        s_m = []
        s_ho = []
        s_ha = []

        self.fn = []
        self.j = 1
        self.k = 0
        
        #getting the necessary filenames
        for filename in sorted(glob.glob('../../Machine Annotate/*.xlsx'), key = numericalSort):
            file = str(filename).rstrip(".xlsx").split('/')[-1]
        #             self.windowTitle = file
            self.fn.append(filename)
        
        #for each filename, get cropped photos of each method with or without traced pattern of facial features
        for filename in sorted(glob.glob('../../Photos/*.png'), key = numericalSort):
        #     print(filename)
            ls = filename.split("_")[-1]
        #     print(ls)
            name = filename.split("/")[-1].split(".")[-2].split("_")[-2]

        #     print(name)
            if (m or m1) in ls:
                s_m.append(filename)

            elif (ho or ho1) in ls:
                s_ho.append(filename)

            elif (ha or ha1) in ls:
                s_ha.append(filename)
#             self.k +=1
#         print(i)

        print(len(s_m))
        # print(s_m)
        print(len(s_ho))
        print(len(s_ha))

        # print(i)
        s_m1 = []
        s_ho1 = []
        s_ha1 = []
        i = 0

        #Filtering the cropped photos with traced patterns (if available)
        for i in range(len(s_m)):
        #     print(i)
            ls = s_m[i].split("_")[-1]
        #     print(ls)
            if key in ls:
                s_m1.append(s_m[i])
            if '1' in ls:
                s_m1.append(s_m[i])

        # print(s_m1)
        print(len(s_m1))

        for i in range(len(s_ho)):
        #     print(i)
            ls = s_ho[i].split("_")[-1]
        #     print(ls)
            if key in ls:
                s_ho1.append(s_ho[i])
            if '1' in ls:
                s_ho1.append(s_ho[i])

        # print(s_ho1)
        print(len(s_ho1))

        for i in range(len(s_ha)):
        #     print(i)
            ls = s_ha[i].split("_")[-1]
        #     print(ls)
            if key in ls:
                s_ha1.append(s_ha[i])
            if '1' in ls:
                s_ha1.append(s_ha[i])

        # print(s_m1)
        print(len(s_ha1))

        self.f= []
        i = 0 
        for i in range(len(s_ha1)):
            self.f.append(s_m1[i])
            self.f.append(s_ho1[i])
            self.f.append(s_ha1[i])
            
            i+=1

        
        #Labeling the image with filename, #, and images left to annotate
        self.label_count = QLabel(self)
        self.label_count.setText("Image #" + str(self.j) + ": " + str(self.fn[self.k].rstrip(".xlsx").split('/')[-1]))
        self.label_count.move(100,10)
        self.label_count.resize(280,40)
        
        self.count_down = 400 - self.j
        self.label_countDown = QLabel(self)
        self.label_countDown.setText("Images left to annotate: " + str(self.count_down))
        self.label_countDown.move(530,10)
        self.label_countDown.resize(200,40)
        
        
        self.i = 0
        #Placing cropped image for MTCNN
        f_mapped_m = self.f[self.i]
        self.file = str(f_mapped_m).rstrip(".png").split('/')[-1] 
        print("Mapped MTCNN: " + str(self.file))
        self.label_m = QLabel(self)
        self.label_m.setText("MTCNN cropped: ")
        self.label_m.move(130,50)
        self.label_m.resize(200,40)

        self.labelM = QLabel(self)
        self.pixmap_m = QPixmap(f_mapped_m).scaled(300,300, Qt.KeepAspectRatio)
        self.labelM.setPixmap(self.pixmap_m)
        self.labelM.resize(self.pixmap_m.width(),self.pixmap_m.height())
        self.labelM.move(60,100)

        
        #Placing cropped image for HOG
        f_mapped_ho = self.f[self.i + 1]
        self.file = str(f_mapped_ho).rstrip(".png").split('/')[-1]
        print("Mapped HOG: " + str(self.file))
        self.label_ho = QLabel(self)
        self.label_ho.setText("HOG cropped: ")
        self.label_ho.move(355,50)
        self.label_ho.resize(200,40)

        self.labelHo = QLabel(self)
        self.pixmap_ho = QPixmap(f_mapped_ho).scaled(300,300, Qt.KeepAspectRatio)
        self.labelHo.setPixmap(self.pixmap_ho)
        self.labelHo.resize(self.pixmap_ho.width(),self.pixmap_ho.height())
        self.labelHo.move(280,100)

        ##Placing cropped image for Haar
        f_mapped_ha = self.f[self.i + 2]
        self.file = str(f_mapped_ha).rstrip(".png").split('/')[-1]
        print("Mapped Haar: " + str(self.file))
        self.label_ha = QLabel(self)
        self.label_ha.setText("Haar cropped: ")
        self.label_ha.move(575,50)
        self.label_ha.resize(200,40)

        self.labelHa = QLabel(self)
        self.pixmap_ha = QPixmap(f_mapped_ha).scaled(300,300, Qt.KeepAspectRatio)
        self.labelHa.setPixmap(self.pixmap_ha)
        self.labelHa.resize(self.pixmap_ha.width(),self.pixmap_ha.height())
        self.labelHa.move(500,100)
#                     self.lay.addWidget(self.label)

    #Function to get ranking from clicked button
    def getText(self,b):
        self.b_text = b.text()
        return self.b_text
    
    #Things that happen after a button is clicked
    def save(self, b):
        print("Clicked!")
        
        #Geting the ranking from getText function
        save_rank = self.getText(b)
        
        
        #Setting up to save the image name and ranking to a file
        self.file = str(self.f[self.i])
        
        #Check if file already exists: append to text if exist, create new and write over the file if not
        if os.path.exists('ranking.txt'):
            print("Yes")
            with open('ranking.txt',"a") as f:
                f.write("\n\n" + str(self.j) + ". " + str(self.fn[self.k].rstrip(".xlsx").split('/')[-1]) +  "\n" + str(save_rank))
        else:
            print("No")
            
            with open('ranking.txt',"w+") as f:
                f.write("\n" + str(self.j) + ". " + str(self.fn[self.k].rstrip(".xlsx").split('/')[-1]) +  "\n" + str(save_rank))
        
        #Replacing the images with a new set of images after button is clicked
        self.j += 1
        self.k+= 1
        
        self.i += 3
        print(self.i)
        
        self.label_count.setText("Image #" + str(self.j) + ": " + str(self.fn[self.k].rstrip(".xlsx").split('/')[-1]))
        self.count_down = 400 - self.j
        self.label_countDown.setText("Images left to annotate: " + str(self.count_down))
        
         #MTCNN
        f_mapped_m = self.f[self.i]
        self.file = str(f_mapped_m).rstrip(".png").split('/')[-1] 
        print("Mapped MTCNN: " + str(self.file))

        self.pixmap_m = QPixmap(f_mapped_m).scaled(300,300, Qt.KeepAspectRatio)
        self.labelM.setPixmap(self.pixmap_m)
        self.labelM.resize(self.pixmap_m.width(),self.pixmap_m.height())
        self.labelM.move(60,100)


        
        
        #HOG
        f_mapped_ho = self.f[self.i + 1]
        self.file = str(f_mapped_ho).rstrip(".png").split('/')[-1]
        print("Mapped HOG: " + str(self.file))
        


        self.pixmap_ho = QPixmap(f_mapped_ho).scaled(300,300, Qt.KeepAspectRatio)
        self.labelHo.setPixmap(self.pixmap_ho)
        self.labelHo.resize(self.pixmap_ho.width(),self.pixmap_ho.height())
        self.labelHo.move(280,100)

        #Haar
        f_mapped_ha = self.f[self.i + 2]
        self.file = str(f_mapped_ha).rstrip(".png").split('/')[-1]
        print("Mapped Haar: " + str(self.file))
        

        self.pixmap_ha = QPixmap(f_mapped_ha).scaled(300,300, Qt.KeepAspectRatio)
        self.labelHa.setPixmap(self.pixmap_ha)
        self.labelHa.resize(self.pixmap_ha.width(),self.pixmap_ha.height())
        self.labelHa.move(500,100)

#First window that pops up
class MyWindow(QMainWindow):
    def __init__(self):
        super(MyWindow,self).__init__()
        self.win = None
        self.setGeometry(500,300,300,200)
        self.setWindowTitle ("Please tell us your name")
        self.initUI()
        
#     Ask for the annotator's name
    def initUI(self):
        print(os.getcwd())
        os.chdir('./')
        
        self.label = QtWidgets.QLabel(self)
        self.label.setText("Name of annotator: ")
        self.label.move(100,10)
        self.label.resize(200,40)
        
        self.textbox = QLineEdit(self)
        self.textbox.move(50, 70)
        self.textbox.resize(200,40)
        
        if self.textbox.returnPressed.connect(self.show_new_window):
            print("Enter is pressed")
        
        self.ok = QtWidgets.QPushButton(self)
        self.ok.setText("OK")
        self.ok.move(50,130)
        self.ok.setStyleSheet("background-color: blue; border-radius : 50; color: rgb(250, 250, 250)")
        self.ok.clicked.connect(self.show_new_window)
        
#   Create a folder with annotator's name and later store a text file with annotator's rankings
    def show_new_window(self):
        self.close()
        
        #Directory
        self.new_folder = self.textbox.text()
        
        
        p = os.getcwd()
        print(p)
        
        # Parent Directory path
        parent_dir = "./Results/Human Annotator/"
  
        # Path
        path = os.path.join(parent_dir, self.new_folder)
#         os.chdir(path)
        
        # Create the directory
        try:
            os.mkdir(path)
            print("Directory '%s' created successfully" % self.new_folder)
        except OSError as error:
            print("Directory '%s' can not be created" % self.new_folder)
        
        os.chdir(path)
        print(path)
        
#         print(name)
        if self.win is None:
            self.win = AnotherWindow()
            self.win.show()
        
        else:
            self.w = None


def start():
    app = QApplication(sys.argv)
    win = MyWindow()
    win.show()
    sys.exit(app.exec_())
    

In [ ]:
start()

/Users/lalida/ETH/Courses/Semester Project/Face Detection
Enter is pressed


2022-06-21 14:39:51.946 python[68056:7026594] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


/Users/lalida/ETH/Courses/Semester Project/Face Detection
Directory 'Terry' can not be created
./Results/Human Annotator/Terry
789
644
795
401
401
401
Mapped MTCNN: 1_0_0_20170109191725028_mapped MTCNN
Mapped HOG: 1_0_0_20170109191725028_mapped HOG
Mapped Haar: 1_0_0_20170109191725028_mapped Haar
Clicked!
Yes
3
Mapped MTCNN: 1_0_2_20161219200004228_mapped MTCNN
Mapped HOG: 1_0_2_20161219200004228_mapped HOG
Mapped Haar: 1_0_2_20161219200004228_mapped Haar
Clicked!
Yes
6
Mapped MTCNN: 1_0_3_20161220220358441_mapped MTCNN
Mapped HOG: 1_0_3_20161220220358441_mapped HOG
Mapped Haar: 1_0_3_20161220220358441_mapped Haar
Clicked!
Yes
9
Mapped MTCNN: 1_0_4_20170103210812538_mapped MTCNN
Mapped HOG: 1_0_4_20170103210812538_mapped HOG
Mapped Haar: 1_0_4_20170103210812538_mapped Haar
Clicked!
Yes
12
Mapped MTCNN: 1_1_0_20170109190938491_mapped MTCNN
Mapped HOG: 1_1_0_20170109190938491_mapped HOG
Mapped Haar: 1_1_0_20170109190938491_mapped Haar
Clicked!
Yes
15
Mapped MTCNN: 1_1_1_20170109194534193

Mapped Haar: 7_1_1_20170109201647377_mapped Haar
Clicked!
Yes
153
Mapped MTCNN: 7_1_2_20170104005105015_mapped MTCNN
Mapped HOG: 7_1_2_20170104005105015_HOG 1
Mapped Haar: 7_1_2_20170104005105015_mapped Haar
Clicked!
Yes
156
Mapped MTCNN: 7_1_3_20170104222541621_mapped MTCNN
Mapped HOG: 7_1_3_20170104222541621_HOG 1
Mapped Haar: 7_1_3_20170104222541621_mapped Haar
Clicked!
Yes
159
Mapped MTCNN: 8_0_0_20170110225017437_mapped MTCNN
Mapped HOG: 8_0_0_20170110225017437_mapped HOG
Mapped Haar: 8_0_0_20170110225017437_mapped Haar
Clicked!
Yes
162
Mapped MTCNN: 8_0_1_20170113132601079_mapped MTCNN
Mapped HOG: 8_0_1_20170113132601079_HOG 1
Mapped Haar: 8_0_1_20170113132601079_mapped Haar
Clicked!
Yes
165
Mapped MTCNN: 8_0_3_20170104230740713_mapped MTCNN
Mapped HOG: 8_0_3_20170104230740713_mapped HOG
Mapped Haar: 8_0_3_20170104230740713_mapped Haar
Clicked!
Yes
168
Mapped MTCNN: 8_0_4_20170103201939191_mapped MTCNN
Mapped HOG: 8_0_4_20170103201939191_HOG 1
Mapped Haar: 8_0_4_20170103201939191